In [1]:
from metafid.deriv import Pricing, OptionStrategy
from metafid.data import TSETMC
import pandas as pd
import numpy as np
import jdatetime as jdt
import requests
from collections import namedtuple
from itertools import combinations
from operator import add
import finpy_tse as fpy
import psycopg
import re
from CONFIG import DB_PASS as PASS
import inspect
from metafid.ta import RetailInstitutionalPower
from metafid.mfw import DB
from metafid.mfw.deriv import OptionStrategyMFW
import pickle

In [2]:
db = DB(dbname="metafid", user="postgres", pass_=PASS)

In [2]:
ostg_mfw = OptionStrategyMFW(dbname="metafid", user="postgres", pass_=PASS, ua_table="sigma", ostg_table="derivs_optionstrategy", pct_daily_cp=0.3, interval=10)
ostg_mfw.do_job()

Press Ctrl+C to exit


In [13]:
"https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/"

'https://optionstrategiesinsider.com/blog/28-option-strategies-that-all-options-traders-should-know/'

In [15]:
def same_strike_and_ex_date_on_call_put(call, put):
    cols = [i for i in call.columns if not i.startswith("ua_") and i not in ["t", "sigma", "dt", "type"]]
    def cols_(x):
        if (x.endswith("_x")) and (x.startswith("ua_")):
            return x.replace("_x","")
        elif (x.endswith("_x")):
            return "call_" + x.replace("_x","")
        elif (x.endswith("_y")):
            return "put_" + x.replace("_y","")
        else:
            return x
    df = call.merge(put[cols], on=["ua", "strike_price", "ex_date"], how="inner")
    df.columns = list(map(cols_, df.columns))
    return df


In [3]:
class OptionStrategy:
    def __init__(self, call:pd.DataFrame, put:pd.DataFrame, pct_daily_cp:float=None) -> None:
            self.call = call
            self.put = put
            self.pct_daily_cp = pct_daily_cp
            self.rep_df = pd.DataFrame(columns=["strategy","position","ua", "ua_final", "t", "bs", "max_pot_profit", "max_pot_loss", "break_even", "current_profit", "pct_cp", "pct_daily_cp", "evaluation"]) 
    def same_strike_and_ex_date_on_call_put(self):
        cols = [i for i in self.call.columns if not i.startswith("ua_") and i not in ["t", "sigma", "dt", "type"]]
        def cols_(x):
            if (x.endswith("_x")) and (x.startswith("ua_")):
                return x.replace("_x","")
            elif (x.endswith("_x")):
                return "call_" + x.replace("_x","")
            elif (x.endswith("_y")):
                return "put_" + x.replace("_y","")
            else:
                return x
        df = self.call.merge(self.put[cols], on=["ua", "strike_price", "ex_date"], how="inner")
        df.columns = list(map(cols_, df.columns))
        return df
    
    def rep_columns(self, cols):
         return [i for i in cols if i in self.rep_df.columns]
    
    def position(self, df):
        def replace_(t:str, char):
            for i in char:
                t = t.replace(i, "")
            return t
        df[["strategy","position"]] = df.stg.map(str).str.split("(", expand=True)
        df["position"] = df.position.apply(lambda x: replace_(x, ["'", ")"]))
        return df


    def long_straddle(self, df):
        """
        A long straddle strategy is an options strategy that involves buying a call and a put on the same
        underlying asset with the same strike price and expiration date. The strike price is usually at-the-money
        or close to it. The goal of this strategy is to profit from a very strong move in either direction by the
        underlying asset, often triggered by a newsworthy event.

        :param df: with (strike_price, ua_final, call_buy_price, put_buy_price) columns
        :return:maximum loss: net premium received
                maximum profit: unlimited
                lower break-even: strike price – net premium
                upper break-even: strike price  + net premium
        """
        stg = namedtuple("LongStraddle", "buy buy_at buy_ buy_at_")
        df["stg"] = df.apply(lambda x: stg(buy=x["call_option"], buy_at=x["call_sell_price"], buy_=x["put_option"], buy_at_=x["put_sell_price"]),
                             axis=1)
        df = self.position(df)
        df = df.assign(max_pot_loss=-df.call_sell_price - df.put_sell_price)
        df = df.assign(
            lower_break_even=df.strike_price + df.max_pot_loss,
            upper_break_even=df.strike_price - df.max_pot_loss,
        )
        df["current_profit"] = df.apply(
            lambda x: max(
                x["max_pot_loss"],
                abs(x["ua_final"] - x["strike_price"]) + x["max_pot_loss"],
            ),
            axis=1,
        )
        df = df.assign(pct_cp=df.current_profit / (df.call_sell_price + df.put_sell_price) * 100).round(2)
        df = df.assign(pct_daily_cp=df.pct_cp / df.t).round(2)
        df = df[self.rep_columns(df.columns)]

        if self.pct_daily_cp:
            return df[df.pct_daily_cp > self.pct_daily_cp]
        else:
            return df

    

In [12]:
mw = TSETMC().mw()
mw.columns

Index(['isin', 'symbol', 'name', 'time', 'open', 'final', 'close', 'no',
       'volume', 'value', 'low', 'high', 'y_final', 'eps', 'base_vol',
       'unknown1', 'unknown2', 'sector', 'day_ul', 'day_ll', 'share_no',
       'mkt_id', 'ob_depth', 'sell_no', 'sell_vol', 'sell_price', 'buy_price',
       'buy_vol', 'buy_no', 'dt'],
      dtype='object')

In [18]:
mw[mw.symbol=="های وب"].iloc[1]

isin                     IRO1HWEB0001
symbol                         های وب
name          داده گسترعصرنوین-های وب
time                           122959
open                             3160
final                            3264
close                            3320
no                               8191
volume                      225397904
value                    735718681257
low                              3155
high                             3368
y_final                          3225
eps                               682
base_vol                     12000000
unknown1                        14678
unknown2                            1
sector                             73
day_ul                        3450.00
day_ll                        3000.00
share_no                  30000000000
mkt_id                            300
ob_depth                            5
sell_no                             1
sell_vol                         2000
sell_price                       3339
buy_price   

In [ ]:
mw = mw[["symbol"]]

In [4]:
call = ostg_mfw.data().call
put = ostg_mfw.data().put

In [5]:
test = OptionStrategy(call=call, put=put, pct_daily_cp=-1)
mixed_df = test.same_strike_and_ex_date_on_call_put()
mixed_df = mixed_df[(mixed_df.call_sell_price >0) & (mixed_df.put_sell_price >0)]
mixed_df

,call_option,call_name,call_final,call_value,call_ob_depth,call_sell_vol,call_sell_price,call_buy_price,call_buy_vol,dt,...,put_option,put_name,put_final,put_value,put_ob_depth,put_sell_vol,put_sell_price,put_buy_price,put_buy_vol,put_bs
25,ضخود3061,اختیارخ خودرو-1600-1402/03/03,2200,55005000,1,100,2500,1104,36,1402-01-20 23:11:32,...,طخود3061,اختیارف خودرو-1600-1402/03/03,26,0,1,1,31,0,0,0
40,ضخود3061,اختیارخ خودرو-1600-1402/03/03,2200,55005000,2,1,3000,0,0,1402-01-20 23:11:32,...,طخود3061,اختیارف خودرو-1600-1402/03/03,26,0,1,1,31,0,0,0
50,ضملی3026,اختیارخ فملی-7333-1402/03/07,1234,0,2,50,1800,10,1000,1402-01-20 23:11:32,...,طملی3026,اختیارف فملی-7333-1402/03/07,1,0,2,1000,500,1,100,13
51,ضملی3026,اختیارخ فملی-7333-1402/03/07,1234,0,2,50,1800,10,1000,1402-01-20 23:11:32,...,طملی3026,اختیارف فملی-7333-1402/03/07,1,0,3,1000,505,0,0,13
52,ضملی3026,اختیارخ فملی-7333-1402/03/07,1234,0,2,50,1800,10,1000,1402-01-20 23:11:32,...,طملی3026,اختیارف فملی-7333-1402/03/07,1,0,1,1000,497,250,30,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010,ضهای3006,اختیارخ های وب-2998-1402/03/03,530,4048648944,1,100,596,536,50,1402-01-20 23:11:32,...,طهای3006,اختیارف های وب-2998-1402/03/03,103,200705852,2,60,134,71,17,15
1011,ضهای3006,اختیارخ های وب-2998-1402/03/03,530,4048648944,1,100,596,536,50,1402-01-20 23:11:32,...,طهای3006,اختیارف های وب-2998-1402/03/03,103,200705852,1,362,91,79,150,15
1012,ضهای3006,اختیارخ های وب-2998-1402/03/03,530,4048648944,1,100,596,536,50,1402-01-20 23:11:32,...,طهای3006,اختیارف های وب-2998-1402/03/03,103,200705852,4,950,168,65,500,15
1013,ضهای3006,اختیارخ های وب-2998-1402/03/03,530,4048648944,1,100,596,536,50,1402-01-20 23:11:32,...,طهای3006,اختیارف های وب-2998-1402/03/03,103,200705852,3,75,140,70,200,15


In [7]:
test.long_straddle(mixed_df).sort_values("pct_daily_cp")

,ua,ua_final,t,strategy,position,max_pot_loss,current_profit,pct_cp,pct_daily_cp
771,شتران,5187,70,LongStraddle,"buy=ضترا3028, buy_at=1000, buy_=طترا3028, buy_...",-2500,-1687,-67.48,-0.96
278,شستا,1410,24,LongStraddle,"buy=ضستا2012, buy_at=649, buy_=طستا2012, buy_a...",-660,-150,-22.73,-0.95
277,شستا,1410,24,LongStraddle,"buy=ضستا2012, buy_at=649, buy_=طستا2012, buy_a...",-654,-144,-22.02,-0.92
275,شستا,1410,24,LongStraddle,"buy=ضستا2012, buy_at=649, buy_=طستا2012, buy_a...",-655,-145,-22.14,-0.92
279,شستا,1410,24,LongStraddle,"buy=ضستا2012, buy_at=649, buy_=طستا2012, buy_a...",-652,-142,-21.78,-0.91
...,...,...,...,...,...,...,...,...,...
304,شستا,1410,143,LongStraddle,"buy=ضستا6004, buy_at=691, buy_=طستا6004, buy_a...",-710,-165,-23.24,-0.16
310,شستا,1410,143,LongStraddle,"buy=ضستا6004, buy_at=684, buy_=طستا6004, buy_a...",-704,-159,-22.59,-0.16
311,شستا,1410,143,LongStraddle,"buy=ضستا6004, buy_at=684, buy_=طستا6004, buy_a...",-706,-161,-22.80,-0.16
299,شستا,1410,143,LongStraddle,"buy=ضستا6004, buy_at=688, buy_=طستا6004, buy_a...",-707,-162,-22.91,-0.16


In [14]:
ri = RetailInstitutionalPower(ticker="برکت", start_date="1401-01-01", end_date="1402-01-10")
df_ri = ri.ri_power()

In [50]:
df_ri["ticker"] = "برکت"
df_ri = df_ri.replace(np.nan, 0)
df_ri[["r_buyer_power", "r_seller_power", "i_buyer_power", "i_seller_power"]] = df_ri[["r_buyer_power", "r_seller_power", "i_buyer_power", "i_seller_power"]].astype(int)
df_ri = df_ri[["date", "ticker", "volume", "adj_final", "r_buyer_power", "r_seller_power", "i_buyer_power", "i_seller_power"]].rename(columns={"adj_final": "close"})
df_ri

NameError: name 'df_ri' is not defined

In [13]:
db = DB(dbname="metafid", user="postgres", pass_=PASS)
db.insert_data(table="ta_ripower", df=df_ri)

In [6]:
db.drop_all(table="ta_ripower")

In [77]:
ri_df = db.query_all(table="tsedata_section", cols="date_id,symbol_id, no_buy_i,no_buy_r,no_sell_i,no_sell_r,val_buy_i,val_buy_r,val_sell_i,val_sell_r")
ri_df["r_buyer_power"] = ri_df["val_buy_r"] / ri_df["no_buy_r"]
ri_df["r_seller_power"] = ri_df["val_sell_r"] / ri_df["no_sell_r"]
ri_df["i_buyer_power"] = ri_df["val_buy_i"] / ri_df["no_buy_i"]
ri_df["i_seller_power"] = ri_df["val_sell_i"] / ri_df["no_sell_i"]

In [78]:
ri_df_last_date = ri_df[ri_df.date_id == ri_df.date_id.max()]
ri_df_last_date["b_s"] = ri_df_last_date.r_buyer_power/ri_df_last_date.r_seller_power
ri_df_last_date = ri_df_last_date.sort_values(by="b_s", ascending=False).head(20)

/var/folders/wr/p3grj4bj0mg_d17_nj95wc8c0000gn/T/ipykernel_5727/263957106.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ri_df_last_date["b_s"] = ri_df_last_date.r_buyer_power/ri_df_last_date.r_seller_power


In [79]:
ri_df_last_date

,date_id,symbol_id,no_buy_i,no_buy_r,no_sell_i,no_sell_r,val_buy_i,val_buy_r,val_sell_i,val_sell_r,r_buyer_power,r_seller_power,i_buyer_power,i_seller_power,b_s
16919,2023-04-09,PYPD,2,75,8,3225,2420000000,44455575450,42360006700,4515568750,5.927410e+08,1.400176e+06,1.210000e+09,5.295001e+09,423.333106
154636,2023-04-09,TMGD,9,363,6,14082,132308000000,1067317054400,878135933600,321489120800,2.940267e+09,2.282979e+07,1.470089e+10,1.463560e+11,128.790812
155112,2023-04-09,LPRS,0,70,3,2105,0,39823772780,24719750000,15104022780,5.689110e+08,7.175308e+06,NaN,8.239917e+09,79.287337
34677,2023-04-09,RADI,0,15,3,131,0,21381782912,15989708768,5392074144,1.425452e+09,4.116087e+07,NaN,5.329903e+09,34.631244
134075,2023-04-09,MRIN,0,6,2,27,0,13144003650,11362500000,1781503650,2.190667e+09,6.598162e+07,NaN,5.681250e+09,33.201176
156917,2023-04-09,LKAR,3,1024,8,1444,2068989150,428204404990,411493612660,18779781480,4.181684e+08,1.300539e+07,6.896630e+08,5.143670e+10,32.153469
111997,2023-04-09,GESF,0,3,0,74,0,6218142000,0,6218142000,2.072714e+09,8.402895e+07,NaN,NaN,24.666667
100355,2023-04-09,BAHN,34,405,5,147,68002718988,602442801486,660716400000,9729120474,1.487513e+09,6.618449e+07,2.000080e+09,1.321433e+11,22.475251
105737,2023-04-09,LAMI,0,6,1,48,0,6473431020,3857700000,2615731020,1.078905e+09,5.449440e+07,NaN,3.857700e+09,19.798461
112410,2023-04-09,GLOR,0,12,2,51,0,12835211200,9885000000,2950211200,1.069601e+09,5.784728e+07,NaN,4.942500e+09,18.490082


In [80]:
ri_df  = ri_df[ri_df.symbol_id.isin(ri_df_last_date.symbol_id)]


In [81]:

ri_df[["r_buyer_power", "r_seller_power", "i_buyer_power", "i_seller_power"]] = ri_df[["r_buyer_power", "r_seller_power", "i_buyer_power", "i_seller_power"]].round(2)


In [82]:
df_hist =db.query_all(table="tsedata_histprice", cols="date_id,symbol_id,final,volume")
tickers =db.query_all(table="tsedata_ticker", cols="symbol,symbol_far")

In [83]:
df_hist = df_hist.merge(tickers, left_on="symbol_id", right_on="symbol", how="inner")
df = df_hist.merge(ri_df, on=["date_id","symbol_id"], how="inner")

In [84]:
df = df[["date_id", "symbol_far", "volume", "final", "r_buyer_power", "r_seller_power", "i_buyer_power", "i_seller_power"]].rename(columns={"date_id": "date", "symbol_far":"ticker"})


In [85]:

df= df.fillna(0)

In [86]:
db.drop_all(table="ta_ripower")

In [87]:
db.insert_data(table="ta_ripower", df=df)